In [1]:
import pandas
import numpy
import datetime
import feather
# import the lshell fit calc module
import os
import sys
module_path = os.path.abspath(os.path.join('/home/bharat/Documents/code/indsvels/py/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import saps_lshell_vel_map

In [2]:
# Some constants
eventDate = datetime.datetime(2011,7,5)
timeStart = datetime.datetime(2011,7,5,3,0)
timeEnd = datetime.datetime(2011,7,5,7,0)
sapsTime = datetime.datetime(2011,7,5,5,0)
sapsRadList = [ "cvw", "cve","fhw",\
               "fhe", "bks", "wal" ]
selBeam = 7
sapsMlatRange = [ 54., 61. ]#[ 56., 60. ]
sapsVelCutoff = 50.
sapsNpointsCutoff = 0 # per radar per beam
velScale = [ -500, 500 ]
velMagnScale = [ 0, 2000 ]

In [3]:
# read from the feather file
velsDF = feather.read_dataframe('../data/saps-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
velsDF["velMagn"] = numpy.abs(velsDF["vLos"])
# setup some fake vals
velsDF["spwdth"] = 50.
velsDF["pwr"] = 10.
# we dont need fits for all the times
velsDF = velsDF[ (velsDF["date"] >= timeStart) &\
               (velsDF["date"] <= timeEnd) ]
velsDF.head()

,dateStr,timeStr,beam,range,azimCalcMag,vLos,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,velMagn,spwdth,pwr
0,20110705,300,0,0,-7.8204,21.1500,39.2439,-145.2032,12.4690,44.3894,143.4903,40,hok,2011-07-05 03:00:00,-11.5310,21.1500,50.0,10.0
1,20110705,300,0,1,-4.6284,18.5725,39.7470,-145.0795,12.4772,44.9081,143.6445,40,hok,2011-07-05 03:00:00,-11.5228,18.5725,50.0,10.0
2,20110705,300,0,2,-2.9119,19.8473,40.1949,-144.9959,12.4828,45.3684,143.7537,40,hok,2011-07-05 03:00:00,-11.5172,19.8473,50.0,10.0
3,20110705,300,0,3,-1.7818,5.4916,40.6224,-144.9279,12.4874,45.8067,143.8465,40,hok,2011-07-05 03:00:00,-11.5126,5.4916,50.0,10.0
4,20110705,300,0,4,-0.9696,-0.8002,41.0394,-144.8684,12.4913,46.2334,143.9308,40,hok,2011-07-05 03:00:00,-11.5087,0.8002,50.0,10.0


In [4]:
allTimeList = velsDF["date"].unique()
fitResFrames = []
failedDTObjs = []
for timeSel in allTimeList:
    print timeSel
    inpVelDF = velsDF[ (velsDF["date"] == timeSel)\
                     ].reset_index(drop=True)
    lmObj = saps_lshell_vel_map.LshellMap( inpVelDF, timeSel )
    # get locations info for getting good fits
    azimCharDF = lmObj.azim_chars()
    # If no good fit is found discard and store the 
    # datetimes in a list for reference!
    if azimCharDF.shape[0] == 0:
        failedDTObjs.append(timeSel)
        print "no good azims found--->", timeSel
        continue
    # get actual good fits
    goodFitDF = lmObj.get_good_fits(azimCharDF)
    print "good fits", goodFitDF.shape
    # If no good fits are found! exit
    if goodFitDF.shape[0] == 0:
        print "no good fits found--->", timeSel
        continue
    # expand the fitting to cells with no fits
    fitResDF = lmObj.expand_fit_results(goodFitDF)
    print "expanding good fits", fitResDF.shape
    # Set up a few conditions to filter out some data
    # 1) If the number of fits is less than 5 skip
    if fitResDF.shape[0] < 5:
        continue
    # 2) number of good fits should be greater than bad one's
    if (fitResDF[ fitResDF["goodFit"] \
            ].shape[0]*1./fitResDF.shape[0]) <= 0.5:
        print "too many bad fits"
        continue
    # 3) Number of unique MLTs in the fits should be greater
    # than or equal to  (for example, 0, 1)
#     if len( fitResDF["normMlt"].unique().tolist() ) < 2.:
#         print "no large scale observations...."
#         continue
    # Need the datetime col as well
    fitResDF["date"] = timeSel
    fitResFrames.append( fitResDF )
# Merge all the good fit DFs
finFitDF = pandas.concat( fitResFrames )

2011-07-05T03:00:00.000000000


/home/bharat/Documents/code/indsvels/py/saps_lshell_vel_map.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfSel["rndAzim"] = dfSel[self.fitAzmType].round()
/home/bharat/Documents/code/indsvels/py/saps_lshell_vel_map.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  x in dfSel["normMLT"] ]


no good azims found---> 2011-07-05T03:00:00.000000000
2011-07-05T03:02:00.000000000
no good azims found---> 2011-07-05T03:02:00.000000000
2011-07-05T03:04:00.000000000
no good azims found---> 2011-07-05T03:04:00.000000000
2011-07-05T03:06:00.000000000
no good azims found---> 2011-07-05T03:06:00.000000000
2011-07-05T03:08:00.000000000
good fits (1, 8)
expanding good fits (2, 10)
2011-07-05T03:10:00.000000000
good fits (0, 8)
no good fits found---> 2011-07-05T03:10:00.000000000
2011-07-05T03:12:00.000000000
no good azims found---> 2011-07-05T03:12:00.000000000
2011-07-05T03:14:00.000000000
no good azims found---> 2011-07-05T03:14:00.000000000
2011-07-05T03:16:00.000000000
no good azims found---> 2011-07-05T03:16:00.000000000
2011-07-05T03:18:00.000000000
good fits (3, 8)
expanding good fits (8, 10)
too many bad fits
2011-07-05T03:20:00.000000000
no good azims found---> 2011-07-05T03:20:00.000000000
2011-07-05T03:22:00.000000000
no good azims found---> 2011-07-05T03:22:00.000000000
2011-0

good fits (18, 8)
expanding good fits (33, 10)
2011-07-05T06:16:00.000000000
good fits (15, 8)
expanding good fits (32, 10)
too many bad fits
2011-07-05T06:18:00.000000000
good fits (12, 8)
expanding good fits (34, 10)
too many bad fits
2011-07-05T06:20:00.000000000
good fits (11, 8)
expanding good fits (31, 10)
too many bad fits
2011-07-05T06:22:00.000000000
good fits (13, 8)
expanding good fits (30, 10)
too many bad fits
2011-07-05T06:24:00.000000000
good fits (12, 8)
expanding good fits (30, 10)
too many bad fits
2011-07-05T06:26:00.000000000
good fits (9, 8)
expanding good fits (23, 10)
too many bad fits
2011-07-05T06:28:00.000000000
good fits (10, 8)
expanding good fits (22, 10)
too many bad fits
2011-07-05T06:30:00.000000000
good fits (9, 8)
expanding good fits (21, 10)
too many bad fits
2011-07-05T06:32:00.000000000
good fits (10, 8)
expanding good fits (14, 10)
2011-07-05T06:34:00.000000000
good fits (8, 8)
expanding good fits (11, 10)
2011-07-05T06:36:00.000000000
good fits (7

In [5]:
feather.write_dataframe( finFitDF, "../data/lshellFits-" + eventDate.strftime("%Y%m%d") + ".feather" )